In [2]:
import torch 
import torchvision.datasets as datasets
from PIL.ImageShow import show
from PIL.Image import Image
import numpy as np

In [3]:
# Load MNIST data set
train_data = datasets.MNIST(root="./data", train=True, download=True, transform=None)

# Get first image data and target and display image
train_image_zero, train_target_zero = train_data[0]
image_zero_pixels = train_image_zero.getdata()

train_image_zero.show()
print(f"The first label is a {train_target_zero}.")
print(f"There are {len(image_zero_pixels)} pixels in the image.")

The first label is a 5.
There are 784 pixels in the image.


In [21]:
alpha = 0.01              

def w_sum(inputs, weights):
    if len(inputs) != len(weights):
        raise ValueError("Vectors need to be of same len")
    output = 0
    
    for input, weight in zip(inputs, weights):
        output += input * weight
    return output

def neural_network(inputs, weights, target):
    pred = w_sum(inputs, weights)
    error = (target - pred) ** 2
    delta = pred - target
    return pred, error, delta
    
def ele_mul(scalar, vector):
    output = np.zeros(len(vector))

    for i in range(len(vector)):
        output[i] = vector[i] * scalar
    
    return output

def forward_propagation(inputs, weights, target):
    pred, error, delta = neural_network(inputs, weights, target)
    weight_deltas = ele_mul(delta, inputs)                          # Why scale input by delta?         
    weights -= alpha * weight_deltas            # Backward propagtion
    return pred, error

def train(data):
    weights = np.random.uniform(-0.5, 0.5, size=784)

    for i in range(len(train_data)):
        input_data, target = train_data[i]
        input_data = np.array(input_data.getdata()) / 255.0

        pred, error = forward_propagation(input_data, weights, target)

        if i % 1000 == 0: 
                print(f"Iteration {i}: Prediction: {pred}, Target: {target}, Error: {error}")
    return weights

In [22]:
weights = train(train_data)



Iteration 0: Prediction: -1.3998975816896115, Target: 5, Error: 40.95868905611653
Iteration 1000: Prediction: 0.18155187448953952, Target: 0, Error: 0.03296108313066551
Iteration 2000: Prediction: 5.709469947326071, Target: 5, Error: 0.5033476061588583
Iteration 3000: Prediction: 3.996063054729717, Target: 9, Error: 25.03938495224089
Iteration 4000: Prediction: 9.57118552914791, Target: 7, Error: 6.610995025299615
Iteration 5000: Prediction: 7.555041575443091, Target: 7, Error: 0.30807115047034805
Iteration 6000: Prediction: 6.0716490115137765, Target: 6, Error: 0.00513358085090127
Iteration 7000: Prediction: 7.031702453731672, Target: 8, Error: 0.937600138109265
Iteration 8000: Prediction: 4.953331989011729, Target: 0, Error: 24.53549779336689
Iteration 9000: Prediction: 9.041439724074053, Target: 6, Error: 9.25035559517565
Iteration 10000: Prediction: 4.4070609522695925, Target: 3, Error: 1.9798205234018125
Iteration 11000: Prediction: 4.672048003133036, Target: 6, Error: 1.763456505